In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DeepClaimStructured(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=4):
        super(DeepClaimStructured, self).__init__()
        
        # Feature transformation layers
        self.f_layer1 = nn.Linear(input_dim, hidden_dim)
        self.f_layer2 = nn.Linear(hidden_dim, hidden_dim)
        
        # Gating mechanism layers
        self.g_layer1 = nn.Linear(input_dim, hidden_dim)
        self.g_layer2 = nn.Linear(hidden_dim, hidden_dim)
        
        # Interaction layers
        self.interaction1 = nn.Linear(hidden_dim, hidden_dim)
        self.interaction2 = nn.Linear(hidden_dim, hidden_dim)
        
        # Final processing layers
        self.h_layer = nn.Linear(hidden_dim, hidden_dim)
        self.addition_layer = nn.Linear(hidden_dim, hidden_dim)
        
        # Multi-task output layers
        self.deny_output = nn.Linear(hidden_dim, 1)  # Denial probability
        self.reason_output = nn.Linear(hidden_dim, output_dim)  # Denial reason codes
        self.response_time_output = nn.Linear(hidden_dim, 1)  # Response time

    def forward(self, x):
        f1 = F.relu(self.f_layer1(x))
        g1 = torch.sigmoid(self.g_layer1(x))
        gated1 = f1 * g1
        
        f2 = F.relu(self.f_layer2(gated1))
        g2 = torch.sigmoid(self.g_layer2(gated1))
        gated2 = f2 * g2
        
        inter1 = F.relu(self.interaction1(gated2))
        inter2 = F.relu(self.interaction2(inter1))
        
        h = F.relu(self.h_layer(inter2))
        summed = self.addition_layer(h)  # Addition operation in structure
        
        # Multi-task outputs
        deny_pred = torch.sigmoid(self.deny_output(summed))
        reason_pred = F.softmax(self.reason_output(summed), dim=1)
        response_time_pred = self.response_time_output(summed)
        
        return deny_pred, reason_pred, response_time_pred

# Example Usage
input_dim = 50  # Assume 50 features
model = DeepClaimStructured(input_dim)
data_sample = torch.randn(1, input_dim)
deny_pred, reason_pred, response_time_pred = model(data_sample)
print("Denial Probability:", deny_pred.item())
print("Reason Code Probabilities:", reason_pred)
print("Predicted Response Time:", response_time_pred.item())

Denial Probability: 0.48922625184059143
Reason Code Probabilities: tensor([[0.2371, 0.2615, 0.2708, 0.2306]], grad_fn=<SoftmaxBackward0>)
Predicted Response Time: -0.08657534420490265
